In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# load data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# train params

In [3]:
learning_rate = 0.01
max_samples = 400000
batch_size = 128
display_step = 10

# raw img = 28*28
n_input = 28
n_steps = 28
n_hidden = 256
n_classes = 10

# placeholder and variable

In [4]:
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

weights = tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
biases = tf.Variable(tf.random_normal([n_classes]))

# bidirectional lstm network

In [5]:
def BiRNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x= tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_steps)
    
    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell,
                                    lstm_bw_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights) + biases

# loss and accuracy

In [6]:
pred = BiRNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# train and test

In [12]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # train
    while step * batch_size < max_samples:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                 "{:.6f}".format(loss) + ", Training Accuracy= " + \
                 "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    
    # test 
    test_len = 10000
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Test Accuracy:", sess.run(accuracy,
                        feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.952173, Training Accuracy= 0.32812
Iter 2560, Minibatch Loss= 1.181409, Training Accuracy= 0.60156
Iter 3840, Minibatch Loss= 0.920694, Training Accuracy= 0.70312
Iter 5120, Minibatch Loss= 0.719263, Training Accuracy= 0.78125
Iter 6400, Minibatch Loss= 0.466293, Training Accuracy= 0.80469
Iter 7680, Minibatch Loss= 0.641807, Training Accuracy= 0.71875
Iter 8960, Minibatch Loss= 0.376148, Training Accuracy= 0.89844
Iter 10240, Minibatch Loss= 0.322392, Training Accuracy= 0.90625
Iter 11520, Minibatch Loss= 0.272719, Training Accuracy= 0.89062
Iter 12800, Minibatch Loss= 0.314769, Training Accuracy= 0.89062
Iter 14080, Minibatch Loss= 0.322288, Training Accuracy= 0.92188
Iter 15360, Minibatch Loss= 0.192020, Training Accuracy= 0.93750
Iter 16640, Minibatch Loss= 0.211111, Training Accuracy= 0.94531
Iter 17920, Minibatch Loss= 0.357592, Training Accuracy= 0.87500
Iter 19200, Minibatch Loss= 0.254312, Training Accuracy= 0.92188
Iter 20480, Minibatch Loss= 0.22

Iter 162560, Minibatch Loss= 0.020786, Training Accuracy= 1.00000
Iter 163840, Minibatch Loss= 0.042686, Training Accuracy= 0.99219
Iter 165120, Minibatch Loss= 0.023831, Training Accuracy= 0.99219
Iter 166400, Minibatch Loss= 0.045751, Training Accuracy= 0.99219
Iter 167680, Minibatch Loss= 0.023148, Training Accuracy= 1.00000
Iter 168960, Minibatch Loss= 0.029187, Training Accuracy= 1.00000
Iter 170240, Minibatch Loss= 0.037081, Training Accuracy= 0.98438
Iter 171520, Minibatch Loss= 0.009136, Training Accuracy= 1.00000
Iter 172800, Minibatch Loss= 0.034034, Training Accuracy= 0.99219
Iter 174080, Minibatch Loss= 0.034774, Training Accuracy= 0.99219
Iter 175360, Minibatch Loss= 0.019742, Training Accuracy= 0.98438
Iter 176640, Minibatch Loss= 0.047146, Training Accuracy= 0.97656
Iter 177920, Minibatch Loss= 0.056071, Training Accuracy= 0.98438
Iter 179200, Minibatch Loss= 0.020149, Training Accuracy= 1.00000
Iter 180480, Minibatch Loss= 0.049473, Training Accuracy= 0.98438
Iter 18176

Iter 322560, Minibatch Loss= 0.047796, Training Accuracy= 0.99219
Iter 323840, Minibatch Loss= 0.006947, Training Accuracy= 1.00000
Iter 325120, Minibatch Loss= 0.010441, Training Accuracy= 1.00000
Iter 326400, Minibatch Loss= 0.034220, Training Accuracy= 0.99219
Iter 327680, Minibatch Loss= 0.006773, Training Accuracy= 1.00000
Iter 328960, Minibatch Loss= 0.006554, Training Accuracy= 1.00000
Iter 330240, Minibatch Loss= 0.031586, Training Accuracy= 0.99219
Iter 331520, Minibatch Loss= 0.002839, Training Accuracy= 1.00000
Iter 332800, Minibatch Loss= 0.030571, Training Accuracy= 0.99219
Iter 334080, Minibatch Loss= 0.011592, Training Accuracy= 1.00000
Iter 335360, Minibatch Loss= 0.054505, Training Accuracy= 0.99219
Iter 336640, Minibatch Loss= 0.042750, Training Accuracy= 0.99219
Iter 337920, Minibatch Loss= 0.028075, Training Accuracy= 0.99219
Iter 339200, Minibatch Loss= 0.028460, Training Accuracy= 0.99219
Iter 340480, Minibatch Loss= 0.006896, Training Accuracy= 1.00000
Iter 34176